In [2]:
import gradio as gr
from datetime import datetime
import pandas as pd
import numpy as np
import pickle
from sklearn.preprocessing import StandardScaler

In [5]:
X_train_full = pd.read_csv("../data/X_train_full.csv")

model_path = "../models/model1.pkl"

with open(model_path, 'rb') as file:
        clf = pickle.load(file)

def clean_dataset(data, X_train_columns):
    data = data.where(pd.notnull(data), None)
    data.fillna(np.nan, inplace=True)
    
    data_scaler = data.copy()
    categorical_cols = ['Brand', 'Sub-brand', 'Product Line', 'Model', 'Version', 'Height', 'Collaboration', 'Color(s)']
    numerical_cols = ['Retail Price', 'Shoe Size']
    
    data.drop(columns=["Buyer Region"], inplace=True, errors='ignore')
    
    data['Order Date'] = pd.to_datetime(data['Order Date'])
    data['Release Date'] = pd.to_datetime(data['Release Date'])
    
    data['Order Year'] = data['Order Date'].dt.year
    data['Order Month'] = data['Order Date'].dt.month
    data['Order Day'] = data['Order Date'].dt.day
    data['Order Day of Week'] = data['Order Date'].dt.dayofweek
    
    data['Release Year'] = data['Release Date'].dt.year
    data['Release Month'] = data['Release Date'].dt.month
    data['Release Day'] = data['Release Date'].dt.day
    data['Release Day of Week'] = data['Release Date'].dt.dayofweek
    
    data['Order Date'] = data['Order Date'].astype(np.int64) // 10**9
    data['Release Date'] = data['Release Date'].astype(np.int64) // 10**9
    
    data = pd.get_dummies(data, columns=categorical_cols, drop_first=True)
    
    scaler = StandardScaler()
    scaler.fit(data_scaler[numerical_cols])
    data[numerical_cols] = scaler.transform(data[numerical_cols])
    
    for col in X_train_columns:
        if col not in data.columns:
            data[col] = 0
    data = data[X_train_columns]

    return data

def predict_shoe_data(
    retail_price, order_date, release_date, shoe_size, buyer_region, brand, sub_brand,
    product_line, model, version, height, collaboration, color
    ):
    if not retail_price or not release_date or not shoe_size or not brand:
        return "Error: Por favor completa todos los campos obligatorios."

    data = {
        "Retail Price": [retail_price],
        "Order Date": [order_date],
        "Release Date": [release_date],
        "Shoe Size": [shoe_size],
        "Buyer Region": [buyer_region],
        "Brand": [brand],
        "Sub-brand": [sub_brand],
        "Product Line": [product_line],
        "Model": [model],
        "Version": [version],
        "Height": [height],
        "Collaboration": [collaboration],
        "Color(s)": [color],
    }
    df = pd.DataFrame(data)
    
    df_cleaned = clean_dataset(df, X_train_full.columns)
    
    try:
        prediction = clf.predict(df_cleaned)
        return f"Predicción: ${prediction[0]:.2f}"
    except Exception as e:
        return f"Error en la predicción: {e}"

states_usa = [
    "Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado", "Connecticut", "Delaware",
    "Florida", "Georgia", "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa", "Kansas", "Kentucky", "Louisiana",
    "Maine", "Maryland", "Massachusetts", "Michigan", "Minnesota", "Mississippi", "Missouri", "Montana",
    "Nebraska", "Nevada", "New Hampshire", "New Jersey", "New Mexico", "New York", "North Carolina",
    "North Dakota", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Rhode Island", "South Carolina",
    "South Dakota", "Tennessee", "Texas", "Utah", "Vermont", "Virginia", "Washington", "West Virginia",
    "Wisconsin", "Wyoming"
]
brands = ["Adidas", "Nike"]
sub_brands = [None, "Yeezy", "Air Jordan"]
product_lines = [None, "Boost", "Air Max", "Air Presto", "Air VaporMax", "Air Force 1", "Blazer Mid", "Zoom Fly", 
                 "React Hyperdunk", "Hyperdunk", "Mercurial", "Retro", "Flyknit"]
models = [None, 350, 1, 90, 97]
versions = [None, "V2", "2pto"]
heights = [None, "High", "Medium", "Low"]
collaborations = [None, "off-white"]
colors = [
    None, 'Beluga', 'Core Black Copper', 'Core Black Green', 'Core Black Red', 'Core Black White',
    'Cream White', 'Zebra', 'Moonrock', 'Pirate Black', 'Oxford Tan', 'Turtledove',
    'Semi Frozen Yellow', 'Blue Tint', 'Black', 'Desert Ore', 'Elemental Rose Queen',
    'All Hallows Eve', 'Grim Reaper', 'Sesame', 'Wolf Grey', 'Menta', 'Black Silver',
    'Pink', 'Volt', 'Butter', 'Static', 'Static Reflective', 'Chicago', 'University Blue',
    'White', 'Black-White', 'Black-Silver', 'Total-Orange'
]

iface = gr.Interface(
    fn=predict_shoe_data,
    inputs=[
        gr.Number(label="Retail Price", precision=2),
        gr.Textbox(label="Order Date", value=str(datetime.today().date()), placeholder="YYYY-MM-DD"),
        gr.Textbox(label="Release Date", placeholder="YYYY-MM-DD"),
        gr.Number(label="Shoe Size"),
        gr.Dropdown(label="Buyer Region", choices=[None] + states_usa, multiselect=False),
        gr.Dropdown(label="Brand", choices=brands),
        gr.Dropdown(label="Sub-brand", choices=sub_brands),
        gr.Dropdown(label="Product Line", choices=product_lines),
        gr.Dropdown(label="Model", choices=models),
        gr.Dropdown(label="Version", choices=versions),
        gr.Dropdown(label="Height", choices=heights),
        gr.Dropdown(label="Collaboration", choices=collaborations),
        gr.Dropdown(label="Color(s)", choices=colors),
    ],
    outputs="text",
    title="Shoe Product Prediction Form",
    description="Complete los campos para predecir el resultado usando el modelo."
)

iface.launch()

* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
